### IO example for  phenol atomic positions optimization (in a box) using Conquest
> The objective is to use ASE for pre-process (building the **I**nput files) and
> post-process (having the **O**utput file) a Conquest calculation. 
> The calculation *must* be run at hand,
> that is, the calculation is not launched by ASE.

<div class="alert alert-block alert-danger">
    <b>WARNING:</b> IO example requirements
</div>    

> `ASE version = 3.20.0b1` including `Conquest`


In [ ]:
import os
import shutil
import subprocess
import matplotlib.pyplot as plt

from distutils.spawn import find_executable
from numpy import amax, amin, matmul, exp, pi, array, zeros, sqrt
from scipy.constants import Rydberg

from ase.build import bulk
from ase.calculators.conquest import Conquest
from ase.io.conquest_f_rework_output_ase import read_conquest, read_conquest_out
from ase.io.conquest_f_rework_output_ase import Conquest_orthorhombic_check
from ase.units import Rydberg

from ase.dft.bandgap import bandgap
from ase.io import read

from ase import Atoms
from ase.spacegroup import crystal

from cq_ase_external_lib import print_struct_data
from cq_ase_external_lib import print_occ, get_gapwind
from cq_ase_external_lib import cq_postproc_xsf, cq_print_output_data


In [ ]:
%%bash 
ase --version

In [ ]:
#### Check if visualisation tools are installed

In [ ]:
# Add exe name 
cmd_vis = {'xcrysden' : False}

for cmd, state in cmd_vis.items():
    if ( not shutil.which(cmd) ):
        print('{} not found'.format(cmd))
    else:
        print('{} found'.format(cmd))   
        cmd_vis[cmd] = True
        
# For MacOSX add the path of the name of the app
cmd_app = {'VESTA'      : '/Applications/VESTA/VESTA.app/Contents/MacOS/VESTA',
           'VMDLauncher': '/Applications/VMD_1.9.2.app/Contents/MacOS/VMDLauncher'}

for cmd, path in cmd_app.items():
    if ( not find_executable(path) ):
        print('{} not found'.format(cmd))
    else:
        print('{} found'.format(cmd))   

#### Define Conquest environment

In [ ]:
os.environ['ASE_CONQUEST_COMMAND'] = 'mpirun -np 4 /Users/lioneltruflandier/CONQUEST-release-f-rework-output-ase/src/Conquest'
os.environ['CQ_PP_PATH'] = '/Users/lioneltruflandier/Conquest-develop-outdated/pseudo-and-pao'
os.environ['CQ_GEN_BASIS_CMD'] = '/Users/lioneltruflandier/CONQUEST-release-develop/tools/BasisGeneration/MakeIonFiles'

#### Directory for storing calculation files

In [ ]:
working_directory = 'cq_example_phenol_optpos_io'

# Test if `working_directory` exists ? If not create it
if ( not os.path.isdir(working_directory) ):
    os.makedirs(working_directory)

####  Build ASE `Atoms` for C$_6$H$_{5}$OH in a box

- Load structure of C$_6$H$_{5}$OH from `PDB` file 

In [ ]:
struct = read('./struct/phenol.pdb',0)

print('n atoms = {} ?'.format(len(struct)))

- Cell parameters in Ang. and deg. [a, b, c, alpha, beta, gamma]. Cell size should be cubic $8\times8\times8$ Ang.

In [ ]:
a = 10.0
b = a
c = a
alpha = 90.0
beta  = 90.0
gamma = 90.0

cell = [a,b,c,alpha,beta,gamma]


- Generate the `Atoms`
 object and store in `struct` 

In [ ]:
struct = Atoms(struct,cell=cell,pbc=[True,True,True])

# Conquest can only handle orthorhombic cells
struct = Conquest_orthorhombic_check(struct,verbose=True)

- Extract and print main structural data from input structure `struct`

In [ ]:
print_struct_data(struct,verbose=1)

- Save input `struct` in `CIF` and `XSF` format 

In [ ]:
struct.write(working_directory+'/input.xsf')
struct.write(working_directory+'/input.cif')

- run `VESTA` to check input structure (if possible)

In [ ]:
if ( 'VESTA' in cmd_app ):
    subprocess.run([cmd_app['VESTA'], working_directory+'/'+'input.cif'])

#### Setup Conquest atomic basis set

In [ ]:
basis = {
         'O' : { 'gen_basis'            : True,
                 'basis_size'           : 'small',
                 'pseudopotential_type' : 'hamann',
                 'xc'                   : 'PBE' },             
         'H' : { 'gen_basis'            : True,
                 'basis_size'           : 'small',
                 'pseudopotential_type' : 'hamann',
                 'xc'                   : 'PBE' },
         'C' : { 'gen_basis'            : True,
                 'basis_size'           : 'small',
                 'pseudopotential_type' : 'hamann',
                 'xc'                   : 'PBE' },         
          }

#basis = {
#         'O' : { 'file' : 'O.ion', 'pseudopotential_type' : 'siesta'},             
#         'C' : { 'file' : 'C.ion', 'pseudopotential_type' : 'siesta'},             
#         'H' : { 'file' : 'H.ion', 'pseudopotential_type' : 'siesta'}             
#}

#### Setup calculation using Conquest as calculator

In [ ]:
cutoff  =  90.0
kpoints = [1,1,1]
fxc     = 'PBE'

conquest_flags = {'IO.WriteOutToASEFile': True}

# Flags for atomic positions optimisation
conquest_flags.update({'AtomMove.TypeOfRun'   : 'sqnm',  # optimization algorithm
                       'AtomMove.MaxForceTol' :  5e-3,   # max Force component in Ha/bohr                       
                       'AtomMove.ReuseDM'     :  True,    
                       'AtomMove.AppendCoords':  True,
                       }) 

calc = Conquest(directory      = working_directory,
                grid_cutoff    = cutoff,
                self_consistent= True,
                xc    = fxc,
                basis = basis,
                kpts  = kpoints,
                nspin = 1,
                **conquest_flags)

struct.calc = calc

#### Generate Conquest input and write

- input file `Conquest_input` is stored in `./working_directory`

In [ ]:
struct.calc.write_input(struct)

<div class="alert alert-block alert-danger">
    <b>WARNING:</b> you must have run the calculation somewhere and store the output files in the working directory
    </b>
</div>

#### Post-process `trajectory.xsf` file generated by Conquest

In [ ]:
traj_file= 'trajectory.xsf'
traj_file=  working_directory+'/'+traj_file

if ( os.path.isfile(traj_file) ):
    # cq_postproc_xsf output last structure from xsf file
    struct_last = cq_postproc_xsf(traj_file,len(struct))
else:
    print('WARNING: {} not found !'.format(traj_file))

- Save ouput `struct_last` in `CIF` and `XSF` format 

In [ ]:
struct_last.write(working_directory+'/output.cif')
struct_last.write(working_directory+'/output.xsf')

- run `xcrysden` to visualize the optimization movie (if possible)

In [ ]:
#if ( cmd_vis['xcrysden'] and os.path.isfile(traj_file) ):
#    subprocess.run(['xcrysden', '--axsf', traj_file])

#### Read Conquest output results and print the data

In [ ]:
struct.calc.read_results(struct)

cq_print_output_data(struct,2)

#### Read last structure from Conquest  `coord_next.dat` 

In [ ]:
last_struct='coord_next.dat'
last_struct=working_directory+'/'+last_struct

if( os.path.isfile(last_struct) ):

    struct_last = read_conquest(last_struct, fractional=True, atomic_order=['H','C','O'])
    
    struct_last.write(working_directory+'/output_.cif')
    struct_last.write(working_directory+'/output_.xsf')

#### Extract and print main structural data from optimized structure `last_struct`

In [ ]:
#print_struct_data(struct_last,verbose=1)

#### Extract Fermi energy, band gap and energy levels

In [ ]:
# Fermi energy as evaluated in Conquest code 'Fermi energy for spin'
fermi_energy = calc.get_fermi_level()
eigenvalues  = calc.get_eigenvalues()

print('Total electronic energy = {:14.6f} eV'.format(struct.calc.results['energy']))
print('Fermi energy            = {:14.6f} eV'.format(fermi_energy))
print_occ(eigenvalues,fermi_energy,0.01,method='aufbau',label='G')

# Evaluate band gap
gap, p1, p2  = bandgap(struct.calc,efermi=fermi_energy,direct=True)